# Performance Test - Deep Seek

In [1]:
import os
import json
import time
from pathlib import Path
from datetime import datetime
import re, sys, subprocess, gc

import pandas as pd
import numpy as np
import torch
import transformers
from transformers import AutoConfig, GenerationConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(model_name)
gen_config = GenerationConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

torch_dtype = torch.bfloat16
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import sys, gc
import torch

def free_mem():
    if hasattr(sys, 'last_traceback'):
        sys.last_traceback.tb_next = None
    gc.collect()
    torch.cuda.empty_cache()
free_mem()

def print_cuda_mem():
    total_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
    cached_memory = torch.cuda.memory_reserved(0) / (1024 ** 3)
    available_memory = total_memory - cached_memory 
    print(f"Available Memory (GB, approx.): {available_memory:.2f}")
print_cuda_memory()

NameError: name 'print_cuda_memory' is not defined

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch_dtype,
                                             attn_implementation="flash_attention_2",
                                             # quantization_config=quantization_config,
                                             device_map="auto")
print_cuda_memory()
print(model.dtype)

In [3]:
questions = pd.read_csv('./train.csv')

In [5]:
tokenizer_max_length = 512
max_length = 1800


tool_instruction = " The answer should be given as a non-negative modulo 1000."
tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

messages = [{
   "role": "user", 
   "content": questions.iloc[0]['problem'] + tool_instruction
}]
query_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False
)
start_time = time.time()
tokenized = tokenizer(query_prompt, 
                      return_tensors="pt",
                      truncation=True,
                      max_length=tokenizer_max_length)
input_length = len(tokenized['input_ids'][0])
print(f'{input_length=}')
tokenized = {key: value.to('cuda') for key, value in tokenized.items()}
raw_output = model.generate(**tokenized, 
                            max_length=max_length,
                            do_sample=True,
                            temperature=0.9,
                            num_return_sequences=4,
                            generation_config=gen_config)
out_list = []
for i in range(len(raw_output)):
    print(f'{len(raw_output[i][input_length:])=}')
    out_list.append(tokenizer.decode(raw_output[i][input_length:], skip_special_tokens=True))
print(time.time() - start_time)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


input_length=125
len(raw_output[i][input_length:])=1675
len(raw_output[i][input_length:])=1675
len(raw_output[i][input_length:])=1675
len(raw_output[i][input_length:])=1675
64.92562174797058


### Test Results
| n Input tokens | n Output tokens  | dtype  | quantization | double quant | flash-atn-2 | eyeball quality | batch size | time | token/s | 
|----------------|------------------|--------|--------------|--------------|-------------|-----------------|------------|------|---------|
| 125            | 1075             |bfloat16| 4-bit        | yes          | yes         | good            | 8          | 99   | 204     |
| 125            | 1075             |bfloat16| 4-bit        | yes          | no          | good            | 8          | 99   | 129     |
| 125            | 1475             |bfloat16| 4-bit        | yes          | yes         | good            | 4          | 52   | 113     |
| 125            | 1155             |bfloat16| 4-bit        | yes          | no          | good            | 1          | 36   | 32      |
| 125            | 1155             |bfloat16| 4-bit        | no           | no          | good            | 1          | 36   | 39      |
| 125            | 1155             |bfloat16| 8-bit        | yes          | no          | good            | 1          | 102  | 11      |
| 125            | 1155             |bfloat16| 8-bit        | no           | no          | good            | 1          | 102  | 11      |
| 125            | OOM              |bfloat16| no           | no           | no          | good            | 8          | OOM  | OOM     |
| 125            | 1475             |bfloat16| no           | no           | no          | good            | 4          | 53   | 111     |
| 125            | 1475             |bfloat16| no           | no           | no          | good            | 2          | 48   | 61      |
| 125            | 1155             |bfloat16| no           | no           | no          | good            | 1          | 36   | 32      |
| 125            | 908              | float16| no           | no           | no          | good            | 1          | 446  | 2       |
| 125            | 552              | float32| no           | no           |  no         |good             | 1          |      |         |

In [5]:
from outlines import models, generate, samplers

In [4]:
sampler = samplers.multinomial(samples=1, temperature=0.9, top_k=10)
outlines_model = models.Transformers(model, tokenizer)
generator = generate.text(outlines_model, sampler)

NameError: name 'samplers' is not defined

In [4]:
tool_instruction = " The answer should be given as a non-negative modulo 1000."
tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

messages = [{
   "role": "user", 
   "content": questions.iloc[0]['problem'] + tool_instruction
}]
query_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False
)

In [11]:
start_time = time.time()
answers = generator([query_prompt] * 4, max_tokens=1600)
token_counts = []
for i in range(len(answers)):
    n_tokens = len(tokenizer(answers[i])['input_ids'])
    token_counts.append(n_tokens)
    print(f"{n_tokens=}")
    
print(f'Ran for {round(time.time() - start_time)}s at {round(sum(token_counts)/(time.time() - start_time), 1)} token/s.')

n_tokens=483
n_tokens=460
n_tokens=407
n_tokens=442
Ran for 34s at 53.1 token/s.


### Using samples=4
n_tokens=494
n_tokens=908
n_tokens=1601
n_tokens=352
Ran for 99s at 33.8 token/s.
n_tokens=652
n_tokens=417
n_tokens=538
n_tokens=572
Ran for 44s at 49.8 token/s.

### Using ptompt *4 
n_tokens=678
n_tokens=380
n_tokens=466
n_tokens=539
Ran for 44s at 47.3 token/s.
n_tokens=483
n_tokens=460
n_tokens=407
n_tokens=442
Ran for 34s at 53.1 token/s.

## vLLM

In [3]:
#pip install vllm

In [4]:
free_mem()
print_cuda_mem()

Available Memory (GB, approx.): 23.64


In [5]:
from vllm import LLM, SamplingParams
vllm = LLM(model=model_name,
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.99,
          swap_space=4,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)
vtokenizer = vllm.get_tokenizer()

WARNING 04-23 15:20:18 config.py:767] Casting torch.bfloat16 to torch.float16.
INFO 04-23 15:20:18 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-23 15:20:18 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='deepseek-ai/deepseek-math-7b-rl', tokenizer='deepseek-ai/deepseek-math-7b-rl', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=fp8_e5m2, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-23 15:20:19 selector.py:51] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-23 15:20:19 selector.py:25] Using XFormers backend.
INFO 04-23 15:20:20 weight_utils.py:177] Using model weights format ['*.safetensors']
INFO 04-23 15:20:23 model_runner.py:104] Loading model weights took 12.8725 GB
INFO 04-23 15:20:24 gpu_executor.py:94] # GPU blocks: 2569, # CPU blocks: 1092


In [6]:
tool_sampling_params = SamplingParams(n=8,
                                      temperature=0.9,
                                      max_tokens=2048,
                                      stop='output')
llm_out = vllm.generate(query_prompt, tool_sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:24<00:00, 24.73s/it]


In [7]:
llm_out[0].outputs[0]

CompletionOutput(index=0, text='```python\nfrom sympy import symbols, solve, sqrt\n\ndef sum_of_squares_distances():\n    """Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin? The answer should be given as a non-negative modulo 1000."""\n    # Symbols\n    k, l, x = symbols(\'k l x\')\n\n    # Equation of the parabola\n    parabola = k*x**2 - 2*k*x + l - 4\n\n    # Equation for the distance between A and B\n    distance = sqrt((x[1] - x[0])**2 + (4 - 4)**2) - 6\n\n    # Solve the system of equations\n    solutions = solve([parabola, distance], (k, l, x))\n\n    # Calculate the sum of the squares of the distances from A and B to the origin\n    sum_of_squares = sum([sqrt(x[0]**2 + 4**2)**2 + sqrt(x[1]**2 + 4**2)**2 for x in solutions])\n\n    # Return the sum modulo 1000\n    return sum_of_squares % 1000\n\

In [60]:
# tool_instruction = " The answer should be given as a non-negative modulo 1000."

n_repetitions = 3
q_ics = []
raw_results = []
total_results = []
total_answers = []

for q_idx in range(len(questions)):
    print(f"\n\n{q_idx}: {questions['problem'].iloc[q_idx]}")
    results = []
    answers = []
    try:
        combined_messages = None
        for rep_idx in range(n_repetitions):
            print(datetime.now().strftime('%H:%M:%S'))
            if rep_idx > (n_repetitions + 1) / 2:
                problem = clean_latex(questions['problem'].iloc[q_idx])
            else:
                problem = questions['problem'].iloc[q_idx] 
            messages = [{
                "role": "user", 
                "content": 'Problem: ' + problem + "\nGenerate Python code to solve the above problem." +
                '\nUnless complex numbers are mentioned in the problem create SymPy symbols with parameter real=True' +
                    '\n```python\n'
            }]
            query_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
            first_output = pipeline(query_prompt, max_new_tokens=1024, do_sample=True, temperature=0.9, return_full_text=False)
            first_output = '```python\n' + first_output[0]['generated_text']
            # print('Shell input: \n' + first_output.split('```')[1][7:] + '\n')
            shell_output = run_python_code(first_output.split('```')[1][7:])
            print(f'{shell_output=}\n')
            with_output = first_output.split('```output')[0] + f'```output\n{shell_output}\n```'
            message = {
                "role": "assistant",
                "content": f'Answer {rep_idx}:\n```python\n' + with_output
            }
            messages.append(message)
            #torch.cuda.empty_cache()
            #gc.collect()
            if combined_messages is None:
                combined_messages = messages
            else: 
                combined_messages.append(message)
                
        combined_messages.append({
            "role": "user",
            "content": ""
                    
        })
        second_prompt = tokenizer.apply_chat_template(combined_messages, tokenize=False).replace('<｜end▁of▁sentence｜>', '')
        # print(f'{second_prompt=}\n')
        second_output = pipeline(second_prompt, max_new_tokens=1024, do_sample=True, temperature=0.1, return_full_text=False)
        print(f'{second_output=}\n')
        
    except Exception as ex:
        print(ex)
        result_output, code_output = -1, -1 
    raw_results.append(second_output)
    result_output, code_output = process_output(second_output)


    results.append(result_output)
    answers.append(code_output)
    q_ics.append(q_idx)
total_results.append(results)
total_answers.append(answers)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.




0: The graph of the equation $9x+223y=2007$ is drawn on graph paper with each square representing one unit in each direction. How many of the $1$ by $1$ graph paper squares have interiors lying entirely below the graph and entirely in the first quadrant?

18:19:20


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


shell_output='Traceback (most recent call last):\n  File "/home/daniel/code/math-ai/code.py", line 21, in <module>\n    result = count_squares()\n             ^^^^^^^^^^^^^^^\n  File "/home/daniel/code/math-ai/code.py", line 11, in count_squares\n    y_intercept = solve(x, y)[0].evalf()\n                  ~~~~~~~~~~~^^^\nIndexError: list index out of range\n'

18:19:54


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...

shell_output='2*sqrt(5) + 13*sqrt(170) + 28*sqrt(785)\n'

18:21:03


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


shell_output='  File "/home/daniel/code/math-ai/code.py", line 2\n    Find the distance of the four points to the focus of the parabola y = x^2.\n         ^^^\nSyntaxError: invalid syntax\n'

18:21:17


--- Logging error ---
Traceback (most recent call last):
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/site-packages/ipykernel_launcher.py",

shell_output='957.750000000000\n'

second_output=[{'generated_text': '```python\nfrom sympy import symbols, solve, N\n\ndef sum_of_distances_intersection_points():\n    """Problem: Let $\\mathcal{P}$ be the parabola in the plane determined by the equation $y = x^2.$  Suppose a circle $\\mathcal{C}$ intersects $\\mathcal{P}$ at four distinct points.  If three of these points are $(-28,784),$ $(-2,4),$ and $(13,169),$ find the sum of the distances from the focus of $\\mathcal{P}$ to all four of the intersection points.\n"""\n    x, y = symbols(\'x y\', real=True)\n\n    # The focus of the parabola y = x^2 is at (0, 1/4)\n    focus_x, focus_y = 0, 1/4\n\n    # The given intersection points\n    points = [(-28, 784), (-2, 4), (13, 169)]\n\n    # The sum of the distances from the focus to the intersection points\n    sum_of_distances = 0\n    for point in points:\n        # Calculate the distance using the distance formula\n        sum_of_distances += ((focus_x - point[0])**2 + (focus_y - p

In [13]:
from sympy import symbols, solve, Eq
def solve_triangle():
    # Define the variables
    a, b, c, pa, cp = symbols('a b c pa cp')
    # From the angle bisector theorem, we have: a/b = (c+a)/(2c)
    # Simplify this to get the ratio of a to b\n 
    ratio = solve(Eq(a/b, (c+a)/(2*c)), a/b)[0]
    # Given that M is the midpoint of AD, we have: pa = 2*cp
    # Substitute this into the ratio to get the ratio of cp to pa
    ratio = ratio.subs(a, 2*cp)
    # Simplify the ratio
    ratio = ratio.simplify()
    # The problem asks for the sum of m and n in the fraction m,
    # where m and n are relatively prime positive integers.
    # The ratio we found is in the form m/n. So, m = cp and n = pa.
    m, n = ratio.as_numer_denom()
    # Calculate m+n\n 
    sum_mn = m + n
    return sum_mn
result = solve_triangle()
print(result)

3*c + 2*cp


In [ ]:
# pd.DataFrame({
#     'question': np.repeat(questions.values, n_repetitions),
#     'sympy_answers': total_answers,
#     'llm_answers': total_results,
#     'raw_answers': raw_results
# })

In [54]:
print(len(np.repeat(questions['problem'].values, n_repetitions)))
print(len(raw_results))
print(len(total_results))
print(len(total_answers))

75
25
8
8


In [65]:
questions['problem'].iloc[7]

'When the expression $4(x^2-2x+2)-7(x^3-3x+1)$ is fully simplified, what is the sum of the squares of the coefficients of the terms?'

In [19]:
total_results

[[-1, -1, -1], [-1, 2, -1], [-1, 1, 1]]

In [20]:
total_answers

[[-1, 995, 995], [2, 2, -1], [-1, 2, 2]]

In [22]:
questions['solution'].values

array(['In general, $\\mathbf{M} \\begin{pmatrix} 1 \\\\ 0 \\end{pmatrix}$ is the first column of $\\mathbf{M}$, and $\\mathbf{M} \\begin{pmatrix} 0 \\\\ 1 \\end{pmatrix}$ is the second column of $\\mathbf{M}.$\n\nTaking $\\mathbf{v} = \\begin{pmatrix} 1 \\\\ 0 \\end{pmatrix},$ we get\n\\[-5 \\begin{pmatrix} 1 \\\\ 0 \\end{pmatrix} = \\begin{pmatrix} -5 \\\\ 0 \\end{pmatrix}.\\]Taking $\\mathbf{v} = \\begin{pmatrix} 0 \\\\ 1 \\end{pmatrix},$ we get\n\\[-5 \\begin{pmatrix} 0 \\\\ 1 \\end{pmatrix} = \\begin{pmatrix} 0 \\\\ -5 \\end{pmatrix}.\\]Therefore,\n\\[\\mathbf{M} = \\boxed{\\begin{pmatrix} -5 & 0 \\\\ 0 & -5 \\end{pmatrix}}.\\]',
       'We have that\n\\begin{align*}\nak^3 + bk^2 + ck + d &= 0, \\\\\nbk^3 + ck^2 + dk + a &= 0.\n\\end{align*}Multiplying the first equation by $k,$ we get\n\\[ak^4 + bk^3 + ck^2 + dk = 0.\\]Subtracting the equation $bk^3 + ck^2 + dk + a = 0,$ we get $ak^4 = a.$  Since $a$ is nonzero, $k^4 = 1.$  Then $k^4 - 1 = 0,$ which factors as\n\\[(k - 1)(k + 1)(

In [7]:
propm_prefix = '''Problem:
Find the domain of the expression $\frac{\sqrt{x-2}}{\sqrt{5-x}}$.}

Solution:
The expressions inside each square root must be non-negative. Therefore,
$x-2 \ge 0$, so $x\ge2$, and $5 - x \ge 0$, so $x \le 5$. Also, the denominator
cannot be equal to zero, so $5-x>0$, which gives $x<5$. Therefore, the domain of
the expression is $\boxed{[2,5)}$.
Final Answer: The final answer is $[2,5)$. I hope it is correct.

Problem:
If $\det \mathbf{A} = 2$ and $\det \mathbf{B} = 12,$ then find
$\det (\mathbf{A} \mathbf{B}).$
Solution:
We have that $\det (\mathbf{A} \mathbf{B}) = (\det \mathbf{A})(\det \mathbf{B})
= (2)(12) = \boxed{24}.$
Final Answer: The final answer is $24$. I hope it is correct.
Problem:
Terrell usually lifts two 20-pound weights 12 times. If he uses two 15-pound
weights instead, how many times must Terrell lift them in order to lift the
same total weight?

Solution:
If Terrell lifts two 20-pound weights 12 times, he lifts a total of
$2\cdot 12\cdot20=480$ pounds of weight. If he lifts two 15-pound
weights instead for $n$ times, he will lift a total of $2\cdot15\cdot n=30n$
pounds of weight. Equating this to 480 pounds, we can solve for $n$:
\begin{align*}
30n&=480\\
\Rightarrow\qquad n&=480/30=\boxed{16}
\end{align*}
Final Answer: The final answer is $16$. I hope it is correct.

Problem:
If the system of equations
\begin{align*}
6x-4y&=a,\\
6y-9x &=b.
\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero,
find $\frac{a}{b},$ assuming $b$ is nonzero.

Solution:
If we multiply the first equation by $-\frac{3}{2}$, we obtain
$$6y-9x=-\frac{3}{2}a.$$Since we also know that $6y-9x=b$, we have
$$-\frac{3}{2}a=b\Rightarrow\frac{a}{b}=\boxed{-\frac{2}{3}}.$$
Final Answer: The final answer is $-\frac{2}{3}$. I hope it is correct.

Problem:
'''

question_easy = 'Beth bakes 4, 2 dozen batches of cokies in a week. If these cookies are shared amongst 16 people equally, how many cookies does each person consume?'
question_hard = 'Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?'
postfix = '\nPlease reason step by step, and put your final answer within \boxed{}.'
postfix_2 =  '\nPlease convert this problem into a set of equations'
tokens = tokenizer.encode(question_hard + postfix_2, return_tensors='pt').to('cuda')
out_tokens = model.generate(tokens, max_length=500)
print(tokenizer.decode(out_tokens[0]))

In [34]:
from sympy import symbols, Rational, floor, sqrt
from sympy import solve, And, Le, Ge
def solve_for_prob():
    x = symbols('x')
    # define the polynomial P(x)
    P = x**2 - 3*x - 9
    # define the condition for the problem
    condition = floor(sqrt(P.subs(x, x))) == sqrt(P.subs(x, floor(x)))
    # define the interval
    interval = And(5 <= x, x <= 15)
    # calculate the number of values of x that satisfy the condition in the interval
    valid_count = 0    
    for i in range(5, 16):
        if condition.subs(x, i): 
            valid_count += 1
        # calculate the total number of values of x in the interval
        total_count = 15 - 5 + 1
        # calculate the probability
        probability = Rational(valid_count, total_count)
        # calculate a, b, c, d, e for the fraction form of the probability
        a = 1
        b = 1
        c = 1
        d = -1
        e = 1
        
solve_for_prob()

AttributeError: 'bool' object has no attribute 'subs'

In [ ]:
˘..